In [1]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
import random
from time import time

from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine

In [2]:
def default_args(**kwargs):
    args = {
        "epochs": 10,
        "number_of_clauses": 10,
        "T": 100,
        "s": 1.0,
        "depth": 2,
        "hypervector_size": 32,
        "hypervector_bits": 2,
        "message_size": 256,
        "message_bits": 2,
        "double_hashing": False,
        "noise": 0.01,
        "number_of_examples": 10000,
        "max_included_literals": 4,
    }

    # Override the defaults with any provided keyword arguments
    for key, value in kwargs.items():
        if key in args:
            args[key] = value

    # Convert to an object-like structure
    class Args:
        pass

    args_obj = Args()
    for key, value in args.items():
        setattr(args_obj, key, value)

    return args_obj

args = default_args()


In [ ]:
map_size = 7 # 7 x 7
data_size = 10000
directions = [
    (-1, 0), (1, 0),  # Left and Right
    (0, -1), (0, 1),  # Up and Down
    (-1, 1), (1, -1)  # Upper-left and Lower-right for staggered rows
]

In [3]:
smbols = ['A, 'O', 'B']
graphs_train = Graphs(
    args.number_of_examples,
    symbols = smbols,
    hypervector_size = args.hypervector_size,
    hypervector_bits = args.hypervector_bits
)

In [4]:
for graph_id in range(data_size):
    graphs_train.set_number_of_graph_nodes(graph_id, 49)

In [5]:
graphs_train.prepare_node_configuration()

In [6]:
n_training = 0

node_counter = 1  # Start node counter from 1
for graph_id in range(n_training):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            
            
            for dx, dy in directions:
            neighbor_x = x + dx
            neighbor_y = y + dy
    
            # Check if neighbor exists within bounds
            if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                number_of_outgoing_edges += 1

            # Use the node_counter to create a unique label for each node
            node_label = f'Node {node_counter}'
            graphs_train.add_graph_node(graph_id, node_label, number_of_outgoing_edges)

            node_counter += 1  # Increment node counter


In [7]:
graphs_train.prepare_edge_configuration()

In [8]:
for graph_id in range(10000):
    edge_type = "Plain"
    graphs_train.add_graph_node_edge(graph_id, 'Node 1', 'Node 2', edge_type)
    graphs_train.add_graph_node_edge(graph_id, 'Node 2', 'Node 1', edge_type)

In [9]:
Y_train = np.empty(10000, dtype=np.uint32)
for graph_id in range(10000):
    x1 = random.choice(['A', 'B'])
    x2 = random.choice(['A', 'B'])

    graphs_train.add_graph_node_property(graph_id, 'Node 1', x1)
    graphs_train.add_graph_node_property(graph_id, 'Node 2', x2)

    if x1 == x2:
        Y_train[graph_id] = 0
    else:
        Y_train[graph_id] = 1

graphs_train.encode()

Node 'Node 1' of graph 0 misses edges.
Node 'Node 2' of graph 0 misses edges.
Node 'Node 1' of graph 1 misses edges.
Node 'Node 2' of graph 1 misses edges.
Node 'Node 1' of graph 2 misses edges.
Node 'Node 2' of graph 2 misses edges.
Node 'Node 1' of graph 3 misses edges.
Node 'Node 2' of graph 3 misses edges.
Node 'Node 1' of graph 4 misses edges.
Node 'Node 2' of graph 4 misses edges.
Node 'Node 1' of graph 5 misses edges.
Node 'Node 2' of graph 5 misses edges.
Node 'Node 1' of graph 6 misses edges.
Node 'Node 2' of graph 6 misses edges.
Node 'Node 1' of graph 7 misses edges.
Node 'Node 2' of graph 7 misses edges.
Node 'Node 1' of graph 8 misses edges.
Node 'Node 2' of graph 8 misses edges.
Node 'Node 1' of graph 9 misses edges.
Node 'Node 2' of graph 9 misses edges.
Node 'Node 1' of graph 10 misses edges.
Node 'Node 2' of graph 10 misses edges.
Node 'Node 1' of graph 11 misses edges.
Node 'Node 2' of graph 11 misses edges.
Node 'Node 1' of graph 12 misses edges.
Node 'Node 2' of gra

SystemExit: -1

/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
graphs_test = Graphs(args.number_of_examples, init_with=graphs_train)

for graph_id in range(args.number_of_examples):
    graphs_test.set_number_of_graph_nodes(graph_id, 2)

graphs_test.prepare_node_configuration()

for graph_id in range(args.number_of_examples):
    number_of_outgoing_edges = 1
    graphs_test.add_graph_node(graph_id, 'Node 1', number_of_outgoing_edges)
    graphs_test.add_graph_node(graph_id, 'Node 2', number_of_outgoing_edges)

graphs_test.prepare_edge_configuration()

for graph_id in range(args.number_of_examples):
    edge_type = "Plain"
    graphs_test.add_graph_node_edge(graph_id, 'Node 1', 'Node 2', edge_type)
    graphs_test.add_graph_node_edge(graph_id, 'Node 2', 'Node 1', edge_type)

Y_test = np.empty(args.number_of_examples, dtype=np.uint32)
for graph_id in range(args.number_of_examples):
    x1 = random.choice(['A', 'B'])
    x2 = random.choice(['A', 'B'])

    graphs_test.add_graph_node_property(graph_id, 'Node 1', x1)
    graphs_test.add_graph_node_property(graph_id, 'Node 2', x2)

    if x1 == x2:
        Y_test[graph_id] = 0
    else:
        Y_test[graph_id] = 1

graphs_test.encode()

tm = MultiClassGraphTsetlinMachine(
    args.number_of_clauses,
    args.T,
    args.s,
    depth = args.depth,
    message_size = args.message_size,
    message_bits = args.message_bits,
    max_included_literals = args.max_included_literals
)

for i in range(args.epochs):
    start_training = time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time()

    start_testing = time()
    result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
    stop_testing = time()

    result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

    print("%d %.2f %.2f %.2f %.2f" % (i, result_train, result_test, stop_training-start_training, stop_testing-start_testing))

weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(args.hypervector_size * 2):
            if tm.ta_action(0, i, k):
                if k < args.hypervector_size:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - args.hypervector_size))

        # for k in range(args.message_size * 2):
        #     if tm.ta_action(1, i, k):
        #         if k < args.message_size:
        #             l.append("c%d" % (k))
        #         else:
        #             l.append("NOT c%d" % (k - args.message_size))

        print(" AND ".join(l))

print(graphs_test.hypervectors)
print(tm.hypervectors)
print(graphs_test.edge_type_id)